In [2]:
import argparse
import copy
import csv
import itertools
import math
import os.path
import pdb
import random
import sys
import time
from collections import Counter
from itertools import permutations, product

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from scipy.integrate import odeint, solve_ivp
from scipy.spatial.distance import braycurtis
from scipy.stats import entropy, pearsonr, spearmanr
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import braycurtis
np.set_printoptions(suppress=True, precision=6)

In [3]:
# # DKO Model
def get_batch(ztrn,ptrn,mb_size):
    s = torch.from_numpy(np.random.choice(np.arange(ptrn.size(dim=0), dtype=np.int64), mb_size, replace=False))
    batch_p = ztrn[s,:]
    batch_q = ptrn[s,:]
    batch_t = t[:batch_time]
    return batch_p.to(device), batch_q.to(device)


def loss_bc(p_i,q_i):
    return torch.sum(torch.abs(p_i-q_i))/torch.sum(torch.abs(p_i+q_i))


def process_data(P):
    #Z = P.copy()
    #Z[Z!=0] = 1
    P = P/P.sum(axis=0)[np.newaxis,:]
    #Z = Z/Z.sum(axis=0)[np.newaxis,:]
    
   
    P = P.astype(np.float32)
#     Z = Z.astype(np.float32)

    P = torch.from_numpy(P.T)
    #Z = torch.from_numpy(Z.T)
    return P


class ODEFunc(torch.nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.fcc1 = torch.nn.Linear(N, N)
        self.fcc2 = torch.nn.Linear(N, N)

    def forward(self, y):
        out = self.fcc1(y)
        out = self.fcc2(out)
        return torch.mul(y,torch.abs(out))/torch.sum(torch.mul(y,torch.abs(out)))
    
def train_reptile(max_epochs,mb,LR,ztrn,ptrn,ztst,ptst,zval,pval,zall,pall):
    loss_train = []
    loss_val = []
    qtst = np.zeros((ztst.size(dim=0),N))
    qtrn = np.zeros((zall.size(dim=0),N))
    ltst_pred = np.zeros((ztst.size(dim=0),1))
    ltst_ground = np.zeros((ztst.size(dim=0),1))
    
    func = ODEFunc().to(device)
    optimizer = torch.optim.Adam(func.parameters(), lr=LR)

    Loss_opt = 1
    for e in range(max_epochs):
        optimizer.zero_grad()
        batch_p, batch_q = get_batch(ztrn,ptrn,mb)
        
        # loss of the traning set
        for i in range(mb):
            p_pred = func(batch_p[i]).to(device)
            p_pred = torch.reshape(p_pred,(1,N))
            if i==0:
                loss = loss_bc(p_pred.unsqueeze(dim=0),batch_q[i].unsqueeze(dim=0))
            else:
                loss = loss + loss_bc(p_pred.unsqueeze(dim=0),batch_q[i].unsqueeze(dim=0))
        loss_train.append(loss.item()/mb)


        # validation set
        for i in range(zval.size(dim=0)):
            p_pred = func(zval[i]).to(device)
            p_pred = torch.reshape(p_pred,(1,N))
            if i==0:
                l_val = loss_bc(p_pred.unsqueeze(dim=0),pval[i].unsqueeze(dim=0))
            else:
                l_val = l_val + loss_bc(p_pred.unsqueeze(dim=0),pval[i].unsqueeze(dim=0))
        loss_val.append(l_val.item()/zval.size(dim=0))
        if l_val.item()/zval.size(dim=0)<=Loss_opt:
            Loss_opt = loss_val[-1]
            best_model = copy.deepcopy(func)
        #print('epoch = ',e, 'loss = ', l_val.item()/mb)

        # update the neural network
        func.zero_grad()
        loss.backward()
        optimizer.step()

        if e == max_epochs-1:
            func = copy.deepcopy(best_model)
            if len(ztst.size())==2:
                for i in range(ztst.size(dim=0)):
                    pred_test = func(ztst[i]).to(device)
                    pred_test = torch.reshape(pred_test,(1,N))
                    qtst[i,:] = pred_test.detach().numpy()
                for i in range(zall.size(dim=0)):
                    pred_test = func(zall[i]).to(device)
                    pred_test = torch.reshape(pred_test,(1,N))
                    qtrn[i,:] = pred_test.detach().numpy()
    return loss_train,qtst,qtrn

def ode_act_and_rep(t, x, B, W_act, W_rep, frozen_idx): 
    dxdt = np.zeros_like(x)
    for j in range(len(x)):  # target genes (regulated nodes)
        if j in frozen_idx:
            continue  # frozen targets are not updated

        activation_sum = 0.0
        repression_sum = 0.0
        for i in range(len(x)):  # regulators
            if i in frozen_idx:
                continue  # frozen regulators do not participate in regulation

            if W_act[i, j] != 0:
                activation_sum += W_act[i, j] * (x[i] / (1 + x[i]))
            if W_rep[i, j] != 0:
                repression_sum += W_rep[i, j] * (1 / (1 + x[i]))

        dxdt[j] = -B * x[j] + activation_sum + repression_sum

    dxdt[frozen_idx] = 0.0  # enforce freezing at the end (extra safety)
    return dxdt

In [5]:
N_cells = 500
N_genes = 100

###################### Step 1: Generate double-KO test data ####################
Orignial = pd.read_csv(
    f'./data/Ptrain.csv',
    header=None
).values.astype(np.float32)   # genes × cells

N_genes, N_cells = Orignial.shape
n_samples = 50000 ###randomly select 50000 double-gene KO samples
rng = np.random.default_rng(2026)

nz_per_cell = [np.flatnonzero(Orignial[:, c]) for c in range(N_cells)]
eligible_cells = np.array([c for c in range(N_cells) if nz_per_cell[c].size >= 2], dtype=int)


perturb_matrix = np.zeros((N_genes, n_samples), dtype=np.float32)
records = np.zeros((n_samples, 3), dtype=int)  # cell, g1, g2

seen = set()
k = 0
pbar = tqdm(total=n_samples, desc="Sampling double-KO")

while k < n_samples:
    cell = int(rng.choice(eligible_cells))
    genes = nz_per_cell[cell]
    g1, g2 = rng.choice(genes, size=2, replace=False)
    g1, g2 = int(g1), int(g2)
    if g1 > g2: g1, g2 = g2, g1  # canonicalize ordering to avoid (a,b) and (b,a) duplicates

    key = (cell, g1, g2)
    if key in seen:
        continue
    seen.add(key)

    x = Orignial[:, cell].copy()
    x[g1] = 0.0
    x[g2] = 0.0

    perturb_matrix[:, k] = x
    records[k] = key
    k += 1
    pbar.update(1)

pbar.close()

pd.DataFrame(perturb_matrix).to_csv(
    f'./data/Ptest_doubleKO.csv',
    header=None, index=None
)
pd.DataFrame(records, columns=["cell_index", "gene1_index", "gene2_index"]).to_csv(
    f'./data/Recorder_doubleKO.csv',
    index=False
)

################## Step 2: Predict profiles #################
max_epochs = 5000
device = 'cpu'
batch_time = 100
t = torch.arange(0.0, 100.0, 0.01)

# ---------- Training set ----------
filepath_train = f'./data/Ptrain.csv'
P = np.loadtxt(filepath_train, delimiter=',')  # (N_genes, N_cells)

Z = P.copy()
Z[Z > 0] = 1

pall = process_data(P)
zall = process_data(Z)

number_of_cols = P.shape[1]
random_indices = np.random.choice(number_of_cols, size=int(0.2 * number_of_cols), replace=False)

P_val = P[:, random_indices]
Z_val = Z[:, random_indices]

P_train = P[:, np.setdiff1d(np.arange(number_of_cols), random_indices)]
Z_train = Z[:, np.setdiff1d(np.arange(number_of_cols), random_indices)]

ptrn = process_data(P_train)
pval = process_data(P_val)
ztrn = process_data(Z_train)
zval = process_data(Z_val)

# ---------- Double-KO test set ----------
filepath_test = f'./data/Ptest_doubleKO.csv'
P1 = np.loadtxt(filepath_test, delimiter=',')  # (N_genes, C)
Z1 = P1.copy()
Z1[Z1 > 0] = 1

ptst = process_data(P1)
ztst = process_data(Z1)

M, N = ptrn.shape
print("ztst shape:", ztst.shape)

# Pre-training to select hyperparameters
LR = 0.01
mb = 20

loss_train, qtst, qtrn = train_reptile(
    max_epochs, mb, LR,
    ztrn, ptrn,
    ztst, ptst,
    zval, pval,
    zall, pall
)

np.savetxt(
    f'./data/PredDoubleKO.csv',
    qtst, delimiter=',', fmt='%.6f'
)

np.savetxt(
    f'./data/PPredDouble.csv',
    qtrn, delimiter=',', fmt='%.6f'
)
print("qtst shape:", qtst.shape)

############## Step 3: Compute KO impact ##############
# ---------- Load double-KO records ----------
records = pd.read_csv(f'./data/Recorder_doubleKO.csv')
Perturb_cell_index  = records['cell_index'].astype(int).to_numpy()
Perturb_gene1_index = records['gene1_index'].astype(int).to_numpy()
Perturb_gene2_index = records['gene2_index'].astype(int).to_numpy()

# ---------- Baseline & prediction ----------
Original = np.loadtxt(f'./data/Ptrain.csv', delimiter=',')  # (N_genes, N_cells)

Original_model = np.loadtxt(
    f'./data/PPredDouble.csv',
    delimiter=','
)

predss = np.loadtxt(
    f'./data/PredDoubleKO.csv',
    delimiter=','
)  # (C, N_genes)

N_genes, N_cells = Original.shape

def get_baseline_from_true_matrix(mat_genes_by_cells, cell_id):
    x = mat_genes_by_cells[:, cell_id].copy()
    s = x.sum()
    return x / s if s > 0 else x

def get_baseline_from_model_matrix(model_mat, cell_id, N_genes, N_cells):
    """
    Support two common shapes:
    - (N_cells, N_genes): use model_mat[cell_id, :]
    - (N_genes, N_cells): use model_mat[:, cell_id]
    """
    if model_mat.shape == (N_cells, N_genes):
        x = model_mat[cell_id, :].copy()
    elif model_mat.shape == (N_genes, N_cells):
        x = model_mat[:, cell_id].copy()
    else:
        # Fallback: infer which axis corresponds to N_genes
        if model_mat.shape[1] == N_genes:
            x = model_mat[cell_id, :].copy()
        elif model_mat.shape[0] == N_genes:
            x = model_mat[:, cell_id].copy()
        else:
            raise ValueError(
                f"Original_model shape={model_mat.shape} cannot align with "
                f"(N_genes,N_cells)=({N_genes},{N_cells})"
            )

    s = x.sum()
    return x / s if s > 0 else x

p_dis = []

for i in range(predss.shape[0]):
    c_id = int(Perturb_cell_index[i])

    # Baseline (true unperturbed expression)
    Without_knock = get_baseline_from_true_matrix(Original, c_id)

    # Baseline (model-predicted unperturbed expression)
    Without_knock1 = get_baseline_from_model_matrix(Original_model, c_id, N_genes, N_cells)

    one_pred = predss[i, :]

    # Predicted KO impact (Bray-Curtis distance to baseline)
    p_dis.append(braycurtis(one_pred, Without_knock1))

p_dis = np.asarray(p_dis, dtype=float)

df = pd.DataFrame({
    "gene1_id": Perturb_gene1_index,
    "gene2_id": Perturb_gene2_index,
    "cell_id":  Perturb_cell_index,
    "k_pred":   p_dis
})
print(df)
df.to_csv(f'./results/Impact_results_doubleKO.csv', index=None)

Sampling double-KO: 100%|█████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 27889.51it/s]


ztst shape: torch.Size([500, 100])
qtst shape: (500, 100)
     gene1_id  gene2_id  cell_id    k_pred
0           4        16      425  0.076976
1           8        45      182  0.045585
2          36        84      321  0.055198
3          71        88      352  0.027712
4          10        63      429  0.065817
..        ...       ...      ...       ...
495        19        27      253  0.061564
496        35        78      422  0.053803
497        14        77      476  0.035865
498        57        81      239  0.026167
499        27        40      338  0.110980

[500 rows x 4 columns]
